In [ ]:
# INSTALL AND LOAD PACKAGES ################################
#install.packages("FinCal")
library(datasets) 
library(dplyr)
library(rvest)
library(FinCal)
# Installs pacman ("package manager") if needed
if (!require("pacman")) install.packages("pacman")

# Use pacman to load add-on packages as desired
pacman::p_load(pacman, rio, tidyverse) 






In [ ]:
# Functions WIP
Change <- function(x) {
    Change_Vector <- c()
    for(i in 2:N) {  
        w <- (x[i]-x[i-1])/x[i-1]
        Change_Vector <- append(Change_Vector, w)
    }
    return(Change_Vector)
}

Graphs <- function(x, color, name, change = Change(x)) {
    barplot(change,
    main = c(stock, "% Change in ", name ," by year", sep=""),
    xlab = "Year",
    names.arg = 2:N,
    ylab = c("% Change in ", name, sep=""),
    col = color)
    
    barplot(x,
    main = c(stock, name, " by year", sep=""),
    xlab = "Year",
    names.arg = 1:N,
    ylab = name,
    col = color)
}
turn_to_double <- function(x) {
  x <-gsub(",","",x)
  x <- gsub('[(]',"-",x)
  x <- gsub('[)]',"",x) %>% as.double() %>% rev()
}


#CURRENTLY UNUSED FUNCTIONS

Change_Graph <- function(x, color, name) {
    barplot(x,
    main = c(stock, "% Change in ", name ," by year", sep=""),
    xlab = "Year",
    names.arg = 2:N,
    ylab = c("% Change in ", name, sep=""),
    col = color)
}

Year_by_Year_Graph <- function(x, color, name) {
    barplot(x,
main = c(stock, name, " by year", sep=""),
xlab = "Year",
names.arg = 1:N,
ylab = name,
col = color)
} 

In [ ]:
# Get desired ticker
stock <- toupper(readline(prompt="Enter stock ticker (if 4 letters add .oq to end): "))
# Load data

income_statement <- capture.output(cat("https://www.reuters.com/companies/",stock,"/financials/income-statement-annual",sep=""))
balance_sheet <- capture.output(cat("https://www.reuters.com/companies/",stock,"/financials/balance-sheet-annual",sep=""))
cash_flow <- capture.output(cat("https://www.reuters.com/companies/",stock,"/financials/cash-flow-annual",sep=""))
key_metrics <- capture.output(cat("https://www.reuters.com/companies/",stock,"/key-metrics", sep=""))

income_statement_page <- read_html(income_statement)
balance_sheet_page <- read_html(balance_sheet)
cash_flow_page <- read_html(cash_flow)
key_metrics_page <- read_html(key_metrics)
# Revenue
Revenue <- income_statement_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
r<-1
while (Revenue[r] != "Total Revenue") {r<-r+1}
Revenue <- Revenue[(r+1):(r+5)]
Revenue <- turn_to_double(Revenue)
# Net Income
Net_Income <- income_statement_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
i<-1
while (Net_Income[i] != "Net Income") {i<-i+1}
Net_Income <- Net_Income[(i+1):(i+5)]
Net_Income <- turn_to_double(Net_Income)
# Equity
Equity <- balance_sheet_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
j<-1
while (Equity[j] != "Total Equity") {j<-j+1}
Equity <- Equity[(j+1):(j+5)]
Equity <- turn_to_double(Equity)
# Debt
Debt <- balance_sheet_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
d<-1
while (Debt[d] != "Total Debt") {d<-d+1}
Debt <- Debt[(d+1):(d+5)]
Debt <- turn_to_double(Debt)
# Operating Cash
Operating_Cash <- cash_flow_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
e<-1
while (Operating_Cash[e] != "Cash from Operating Activities") {e<-e+1}
Operating_Cash <- Operating_Cash[(e+1):(e+5)]
Operating_Cash <- turn_to_double(Operating_Cash)
# Total Assets
Total_Assets <- balance_sheet_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
b<-1
while (Total_Assets[b] != "Total Assets") {b<-b+1}
Total_Assets <- Total_Assets[(b+1):(b+5)]
Total_Assets <- turn_to_double(Total_Assets)
# Earnings Per Share
EPS <- key_metrics_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
w<-1
while (EPS[w] != "EPS Basic excl. Extra Items (TTM)") {w<-w+1}
EPS <- EPS[(w+1)]
EPS <- turn_to_double(EPS)
# P/E ratio
PE <- key_metrics_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
q<-1
while (PE[q] != 'P/E excl. Extra Items (TTM)') {q<-q+1}
PE <- PE[(q+1)]
PE <- turn_to_double(PE)
# Free Cash Flow Per Share Data
Free_Cash_Flow_Per_Share <- key_metrics_page %>% html_nodes("tr .TextLabel__regular___2X0ym") %>% html_text()
z<-1
while (Free_Cash_Flow_Per_Share[z] != "Free Cash Flow (Per Share TTM)") {z<-z+1}
Free_Cash_Flow_Per_Share <- Free_Cash_Flow_Per_Share[(z+1)]
Free_Cash_Flow_Per_Share <- turn_to_double(Free_Cash_Flow_Per_Share)


In [ ]:
# Length of data
N <- length(Net_Income)

In [ ]:
# sets frame for graphs
par(mfrow = c(1,2))

Graphs(Revenue, "darkgreen", "Revenue")

In [ ]:
# sets frame for graphs
par(mfrow = c(1,2))
#Creates Graphs
Graphs(Operating_Cash, "green", "Operating Cash")

In [ ]:
# sets frame for graphs
par(mfrow = c(1,2))
Graphs(Debt, "darkred", "Debt")

In [ ]:
# sets frame for graphs
par(mfrow = c(1,2))
Graphs(Net_Income, "yellow", "Net Income")

In [ ]:
# sets frame for graphs
par(mfrow = c(1,2))
Graphs(Equity, "blue", "Equity")

#text(bpEC, 0, round(Change(Equity), digits=2), cex=0.9, font = 2, pos = 3)
#text(bpE, 0, round(Equity), cex=0.9, font = 2, pos = 3)

In [ ]:
# Calculate ROE
ROE <- c()
for(i in 1:N) {
  x <- Net_Income[i]/Equity[i] 
  ROE <- append(ROE,x)
}
# sets frame for graphs
par(mfrow = c(1,2))
bpROE <- barplot(ROE,
main = c(stock, "ROE by year", sep=""),
xlab = "Year",
names.arg = 1:N,
ylab = "ROE",
col = "lightblue")
text(bpROE, 0, round(ROE, digits=2), cex=0.9, font = 2, pos = 3)

# ROIC
ROIC <- c()
for(i in 1:N) {
  x <- Net_Income[i]/(Equity[i] + Debt[i]) 
  ROIC <- append(ROIC,x)
}

bpROIC <- barplot(ROIC,
main = c(stock, "ROIC by year", sep=""),
xlab = "Year",
names.arg = 1:N,
ylab = "ROIC",
col = "lightblue")
text(bpROIC, 0, round(ROIC, digits=2), cex=0.9, font = 2, pos = 3)

In [ ]:
DtoE <- c()
for(i in 1:N) {
  x <- Debt[i]/Equity[i]
  DtoE <- append(DtoE,x)
}
print(cat("Debt to Equity:",DtoE))

DR <- c()
for(i in 1:N) {
  x <- Debt[i]/Total_Assets[i]
  DR <- append(DR,x)
}
print(cat("Debt Ratios:", DR))

In [ ]:
#Valuation
a <- Free_Cash_Flow_Per_Share
for (i in 1:8) {
  a[i+1] <- a[i]+a[i]*.1
}
BuyPrice1 <- sum(a)-a[1]
proj_growth <- readline(prompt = "Enter projected growth: ")
Future_Value <- fv(as.double(proj_growth),10, -EPS)*PE
Present_Value <- pv(.15,10,-Future_Value)
BuyPrice2 <- Present_Value/2
cat("Buy Prices: ",round(BuyPrice1,2),";",round(BuyPrice2,2),".",sep="")


In [ ]:
#Verification
data.frame(Revenue,Debt,Net_Income,Equity,Operating_Cash)
c(EPS,PE,Free_Cash_Flow_Per_Share)